In [1]:
gROOT->ProcessLine(".L common/gallery_includes.h+O");
gROOT->ProcessLine(".L common/larsoftobj_includes.h+O");
gROOT->ProcessLine(".L common/My_ROOT_Utilities.h");

In [2]:
//for space charge
gROOT->ProcessLine(".L $GSL_LIB/libgsl.so");
gROOT->ProcessLine(".L $UBOONECODE_LIB/libuboone_SpaceCharge.so");
gROOT->ProcessLine(".L common/Utils_SpaceCharge.h+O")

(long) 0


In [3]:
//setup our input file and input tag
std::vector<std::string> filenames;
std::string file_path;
TFile *f_output;

TNtuple *nt;
TCanvas canvas("canvas","MyCanvas");

art::InputTag vtx_tag;
art::InputTag pfp_tag;
art::InputTag gen_tag;

//geometry
float X_LENGTH=256.0;
float Y_HALF_HEIGHT=116.5;
float Z_LENGTH=1037.0;

float PROTON_THRESHOLD=0.030;

std::unique_ptr<spacecharge::SpaceChargeMicroBooNE> sce;
CreateSCE(sce);
geo::Vector_t offsets;

bool verbose=false;

In [4]:
%%cpp -d
void PrintDaughters(size_t i_pfp,
                    std::vector<recob::PFParticle> const&,
                    int ntab=0);

void PrintDaughters(size_t i_pfp,
                    std::vector<recob::PFParticle> const& pfp_vec,
                    int ntab)
{
    for(size_t i_d=0; i_d<pfp_vec[i_pfp].NumDaughters(); ++i_d){
        for(int it=0; it<ntab; ++it) std::cout << "\t";
        std::cout << "Found daughter particle id=" << pfp_vec[i_pfp].Daughters()[i_d] 
                  << ", code=" << pfp_vec[pfp_vec[i_pfp].Daughters()[i_d]].PdgCode() << std::endl;
        PrintDaughters(pfp_vec[i_pfp].Daughters()[i_d],pfp_vec,ntab+1);
    }
    return;
}

In [5]:
verbose=false;

//filepath = "/pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/";
filepath = "";
filenames = GetFileNameList("/uboone/data/users/wketchum/numu_devset_reco2_stripped_files/numu_devset_reco2_stripped_filelist.txt",filepath);

f_output = new TFile("/uboone/data/users/wketchum/numu_devset_vtx_ntuple_pnu_tmp.root","RECREATE");

nt = new TNtuple("nt","MyNtuple","nu_x:nu_y:nu_z:nu_e:q2:lep_p:n_p:p_ke:vtx_x:vtx_y:vtx_z:vtx_dist:pfp_primary:pfp_pdg");

vtx_tag = { "pandoraNu" };
pfp_tag = { "pandoraNu" };
gen_tag = { "generator" };


In [6]:
//filenames.size()
//std::cout << "working?" << std::endl;
//for(auto & fname : filenames)
    //std::cout << fname << std::endl;
//filenames.resize(1);

In [7]:
for(gallery::Event ev(filenames) ; !ev.atEnd(); ev.next()) {
    if(verbose)
        std::cout << "Processing " 
                  << "Run " << ev.eventAuxiliary().run() << ", "
                  << "Event " << ev.eventAuxiliary().event() << endl;
    
    auto const& mctruth_handle = ev.getValidHandle<std::vector<simb::MCTruth>>(gen_tag);
    auto const& truth_vec(*mctruth_handle);

    if(truth_vec.size()==0){
        std::cout << "ERROR! truth_vec size is " << truth_vec.size() << std::endl;
        break;
    }
       
    auto const& nu = truth_vec[0].GetNeutrino();
    geo::Point_t nu_position(nu.Nu().Vx(),nu.Nu().Vy(),nu.Nu().Vz());

    if(verbose)
        std::cout << "Neutrino position before sce correction is (" 
                  << nu_position.X() << ","
                  << nu_position.Y() << ","
                  << nu_position.Z() << ")"
                  << std::endl;

    nu_position = nu_position + sce->GetPosOffsets(nu_position);
    if(nu_position.X()<0.0+10.0 || nu_position.X()>X_LENGTH-10.0 ||
       nu_position.Y()<-1*Y_HALF_HEIGHT+20.0 || nu_position.Y()>Y_HALF_HEIGHT-20.0 ||
       nu_position.Z()<0.0 || nu_position.Z()>Z_LENGTH-10.0 )
        continue;

    if(verbose)
        std::cout << "Neutrino position after sce correction is (" 
                  << nu_position.X() << ","
                  << nu_position.Y() << ","
                  << nu_position.Z() << ")"
                  << std::endl;

    float lep_energy = -999, lep_p = -999;
    if(nu.Mode()==0) { //CC
        lep_energy = nu.Lepton().E();
        lep_p = nu.Lepton().P();
    }
    int n_proton=0; float max_proton_ke=0.0;

    for(size_t i_p=0; i_p<truth_vec[0].NParticles(); ++i_p){
        auto const& p = truth_vec[0].GetParticle(i_p);

        if(p.StatusCode()!=1 || p.PdgCode()!=2212) continue;
        
        auto ke = (p.E()-p.Mass());
        if(ke > PROTON_THRESHOLD) ++n_proton;
        if(ke>max_proton_ke) max_proton_ke = ke;        
    }

    auto const& pfp_handle = ev.getValidHandle<std::vector<recob::PFParticle>>(pfp_tag);
    auto const& pfp_vec(*pfp_handle);
    if(verbose)
        std::cout << "There are " << pfp_vec.size() << " PFParticles in this event." << std::endl;

    for (auto const& pfp : pfp_vec){
        if(!pfp.IsPrimary()) continue;
        if(verbose){
            std::cout << "Found primary particle id=" << pfp.Self() << ", code=" << pfp.PdgCode() << std::endl;
            PrintDaughters(pfp.Self(),pfp_vec,1);
        }
    }
    
    
    auto const& vtx_handle = ev.getValidHandle<std::vector<recob::Vertex>>(vtx_tag);
    auto const& vtx_vec(*vtx_handle);
    if(verbose)
        std::cout << "There are " << vtx_vec.size() << " vertices in this event." << std::endl;

    art::FindOneP<recob::Vertex> vtx_per_pfp(pfp_handle,ev,pfp_tag);
    art::Ptr<recob::Vertex> vtxptr;
    double min_dist=9e9; int i_pfp_nu=-1; geo::Point_t pfp_position;
    
    for(size_t i_pfp=0; i_pfp<pfp_vec.size(); ++i_pfp)
    {
        
        vtx_per_pfp.get(i_pfp,vtxptr);
        if(vtxptr.isNonnull()){
            auto const& vtx(*vtxptr);
            if(verbose)
                std::cout << "For pfparticle " << i_pfp << " found vertex at "
                          << "(" << vtx.position().X() << ","
                          << ""  << vtx.position().Y() << ","
                          << ""  << vtx.position().Z() << ")"
                          << std::endl;
            
            if(sqrt((vtx.position()-nu_position).Mag2()) < min_dist){
                min_dist = sqrt((vtx.position()-nu_position).Mag2());
                i_pfp_nu = i_pfp;
                pfp_position = vtx.position();
            }
        }
        
            
    }
    
    nt->Fill(nu_position.X(),nu_position.Y(),nu_position.Z(),
             nu.Nu().E(),nu.QSqr(),lep_p,n_proton,max_proton_ke,
             pfp_position.X(),pfp_position.Y(),pfp_position.Z(),
             min_dist,
             pfp_vec[i_pfp_nu].IsPrimary(),pfp_vec[i_pfp_nu].PdgCode());
    
    
}//end loop over events


Successfully opened file /uboone/data/users/wketchum/numu_devset_reco2_stripped_files/numu_tune3_reco2_stripped_1.root
Closing file, read 1716659 bytes in 420 transactions
Successfully opened file /uboone/data/users/wketchum/numu_devset_reco2_stripped_files/numu_tune3_reco2_stripped_10.root
Closing file, read 1794455 bytes in 361 transactions
Successfully opened file /uboone/data/users/wketchum/numu_devset_reco2_stripped_files/numu_tune3_reco2_stripped_11.root
Closing file, read 1821695 bytes in 353 transactions
Successfully opened file /uboone/data/users/wketchum/numu_devset_reco2_stripped_files/numu_tune3_reco2_stripped_12.root
Closing file, read 1840396 bytes in 361 transactions
Successfully opened file /uboone/data/users/wketchum/numu_devset_reco2_stripped_files/numu_tune3_reco2_stripped_13.root
Closing file, read 1861547 bytes in 372 transactions
Successfully opened file /uboone/data/users/wketchum/numu_devset_reco2_stripped_files/numu_tune3_reco2_stripped_14.root
Closing file, re

Closing file, read 2291668 bytes in 359 transactions
Successfully opened file /uboone/data/users/wketchum/numu_devset_reco2_stripped_files/numu_tune3_reco2_stripped_53.root
Closing file, read 2307313 bytes in 360 transactions
Successfully opened file /uboone/data/users/wketchum/numu_devset_reco2_stripped_files/numu_tune3_reco2_stripped_54.root
Closing file, read 2345928 bytes in 373 transactions
Successfully opened file /uboone/data/users/wketchum/numu_devset_reco2_stripped_files/numu_tune3_reco2_stripped_55.root
Closing file, read 2318440 bytes in 353 transactions
Successfully opened file /uboone/data/users/wketchum/numu_devset_reco2_stripped_files/numu_tune3_reco2_stripped_56.root
Closing file, read 2333848 bytes in 362 transactions
Successfully opened file /uboone/data/users/wketchum/numu_devset_reco2_stripped_files/numu_tune3_reco2_stripped_57.root
Closing file, read 2368718 bytes in 361 transactions
Successfully opened file /uboone/data/users/wketchum/numu_devset_reco2_stripped_fi

In [8]:
f_output->cd();
nt->Write();
f_output->Close();

Todos:

---Double-check space charge correction.

---Store lepton angles and number of final state particles in tree

---

---Understand the fraction of charge in the reconstructed heirarchies relative to the truth deposited. Compare across "well-reconstructed" (close to vertex) and "poorly-reconstructed" (not close to vertex)

---Should look at distance of reconstructed primary to the truth, and not restrict to closest vertex (could be a daughter is super close to the primary too)

